This code is modified version of code from https://www.kaggle.com/pestipeti/pytorch-starter-fasterrcnn-train

In [14]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import time
import copy
from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision
from sklearn.utils import shuffle

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt
from numba import jit

DIR_INPUT = '../input/adl-a1-cropping/cropping_object_detection_data'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

In [15]:
train_df = pd.read_csv(f'{DIR_INPUT}/train_data.csv')
train_df.shape

In [16]:
train_df.head()

In [17]:
train_df['x'] = train_df['x'].astype(np.float)
train_df['y'] = train_df['y'].astype(np.float)
train_df['w'] = train_df['w'].astype(np.float)
train_df['h'] = train_df['h'].astype(np.float)

In [18]:
train_df = shuffle(train_df, random_state=1411)

In [19]:
image_ids = train_df['image_id'].unique()
valid_ids = image_ids[-486:]
train_ids = image_ids[:-486]

In [20]:
valid_df = train_df[train_df['image_id'].isin(valid_ids)]
train_df = train_df[train_df['image_id'].isin(train_ids)]

In [21]:
valid_df.shape, train_df.shape

In [22]:
class DetectionDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]
        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        temp_boxes = records[['x', 'y', 'w', 'h']].values
        boxes = temp_boxes.copy()
        boxes[:, 0] = temp_boxes[:, 0] - temp_boxes[:, 2]/2
        boxes[:, 1] = temp_boxes[:, 1] - temp_boxes[:, 3]/2
        boxes[:, 2] = temp_boxes[:, 0] + temp_boxes[:, 2]/2
        boxes[:, 3] = temp_boxes[:, 1] + temp_boxes[:, 3]/2
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [23]:
# Albumentations
def get_train_transform():
    return A.Compose([
        A.Resize(800,600),
        A.HorizontalFlip(0.3),
        A.RandomBrightnessContrast(p=0.1),
        A.Rotate(limit = 15, p= .4),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


# Create the model

In [24]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [25]:
num_classes = 2  # 1 class (head) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [26]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


In [27]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = DetectionDataset(train_df, DIR_TRAIN, get_train_transform())
valid_dataset = DetectionDataset(valid_df, DIR_TRAIN, get_valid_transform())


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=12,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)
dataloaders = {
    "train": train_data_loader, 
    "val": valid_data_loader,
}

In [28]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

# Sample

In [29]:
sample = train_dataset[0]

In [30]:
sample[0].shape

In [31]:
sampleiter = iter(train_data_loader)

In [32]:
images, targets, image_ids = next(sampleiter)

In [33]:
images, targets, image_ids = next(sampleiter)
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [34]:
boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
sample = images[0].permute(1,2,0).cpu().numpy()

In [35]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

The below code for IoU calculation and precision calculation is taken from https://www.kaggle.com/havinath/object-detection-using-pytorch-training#Obect-detection-using-PyTorch---Training. 

In [38]:
@jit(nopython=True)
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area

@jit(nopython=True)
def find_best_match(gts, pred, pred_idx, threshold = 0.5, form = 'pascal_voc', ious=None) -> int:
    """Returns the index of the 'best match' between the
    ground-truth boxes and the prediction. The 'best match'
    is the highest IoU. (0.0 IoUs are ignored).

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        pred: (List[Union[int, float]]) Coordinates of the predicted box
        pred_idx: (int) Index of the current predicted box
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (int) Index of the best match GT box (-1 if no match above threshold)
    """
    best_match_iou = -np.inf
    best_match_idx = -1

    for gt_idx in range(len(gts)):
        
        if gts[gt_idx][0] < 0:
            # Already matched GT-box
            continue
        
        iou = -1 if ious is None else ious[gt_idx][pred_idx]

        if iou < 0:
            iou = calculate_iou(gts[gt_idx], pred, form=form)
            
            if ious is not None:
                ious[gt_idx][pred_idx] = iou

        if iou < threshold:
            continue

        if iou > best_match_iou:
            best_match_iou = iou
            best_match_idx = gt_idx

    return best_match_idx

@jit(nopython=True)
def calculate_precision(gts, preds, threshold = 0.5, form = 'coco', ious=None) -> float:
    """Calculates precision for GT - prediction pairs at one threshold.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (float) Precision
    """
    n = len(preds)
    tp = 0
    fp = 0
    
    # for pred_idx, pred in enumerate(preds_sorted):
    for pred_idx in range(n):

        best_match_gt_idx = find_best_match(gts, preds[pred_idx], pred_idx,
                                            threshold=threshold, form=form, ious=ious)

        if best_match_gt_idx >= 0:
            # True positive: The predicted box matches a gt box with an IoU above the threshold.
            tp += 1
            # Remove the matched GT box
            gts[best_match_gt_idx] = -1

        else:
            # No match
            # False positive: indicates a predicted box had no associated gt box.
            fp += 1

    # False negative: indicates a gt box had no associated predicted box.
    fn = (gts.sum(axis=1) > 0).sum()

    return tp / (tp + fp + fn)


@jit(nopython=True)
def calculate_image_precision(gts, preds, thresholds = (0.5, ), form = 'coco') -> float:
    """Calculates image precision.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        thresholds: (float) Different thresholds
        form: (str) Format of the coordinates

    Return:
        (float) Precision
    """
    n_threshold = len(thresholds)
    image_precision = 0.0
    
    ious = np.ones((len(gts), len(preds))) * -1
    # ious = None

    for threshold in thresholds:
        precision_at_threshold = calculate_precision(gts.copy(), preds, threshold=threshold,
                                                     form=form, ious=ious)
        image_precision += precision_at_threshold / n_threshold

    return image_precision

In [39]:
iou_thresholds = [x for x in np.arange(0.5, 0.75, 0.1)]

def get_val_iou(predictions, gt_boxes):
    validation_image_precisions = []
    for i in range(len(predictions)):
        preds = predictions[i]["boxes"].data.cpu().numpy().astype(np.int32)
        scores = predictions[i]["scores"].data.cpu().numpy().astype(np.int32)
        preds_sorted_idx = np.argsort(scores)[::-1]
        preds_sorted = preds[preds_sorted_idx]
        gt_box = gt_boxes[i]['boxes'].cpu().numpy().astype(np.int32)
        image_precision = calculate_image_precision(preds_sorted,
                                                    gt_box,
                                                    thresholds=iou_thresholds,
                                                    form='pascal_voc')
        validation_image_precisions.append(image_precision)
    
    val_iou = np.mean(validation_image_precisions)
    return val_iou

# Train

In [40]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.003, weight_decay=0.0005)
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                          mode='max', 
                                                          factor=0.1, 
                                                          patience=3,
                                                          verbose=True)
num_epochs = 25

In [41]:
train_loss_avg =  Averager()
val_iou_avg = Averager()

train_loss_hist = []
val_iou_hist = []

itr = 0
best_iou = 0
best_model_wts = copy.deepcopy(model.state_dict())
best_epoch_num = -1

In [ ]:
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    
    since = time.time()
    
    for phase  in  ["train", "val"]:
        if phase == "train":
            model.train()
            train_loss_avg.reset()
        else:
            model.eval()
            val_iou_avg.reset()
            
        running_loss = 0
        running_iou = 0
        counter = 0
        for images, targets, image_ids in dataloaders[phase]:
            if phase == "train": 
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                optimizer.zero_grad()
        
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                losses.backward()
                optimizer.step()
                
                loss_value = losses.item()
                train_loss_avg.send(loss_value)
                running_loss += loss_value*len(images)
                
                if counter % 50 == 0:
                    print(f"{phase} Iteration #{itr} loss: {loss_value}")
                itr += 1
            
            elif phase == "val":
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                with torch.no_grad():
                    outputs = model(images)
                
                val_iou = get_val_iou(outputs, targets)
                val_iou_avg.send(val_iou)
                running_iou += val_iou*len(images)
                if counter % 50 == 0:
                    print(f"{phase} Iteration #{itr} IoU: {val_iou}")
                    
                
                
            counter += 1
            
        if phase == "train":
            epoch_loss = running_loss/len(dataloaders[phase].dataset)
            train_loss_hist.append(epoch_loss)
        else:
            epoch_iou = running_iou/len(dataloaders[phase].dataset)
            val_iou_hist.append(epoch_iou)
            
            
        if phase == "val" and epoch_iou > best_iou:
            best_iou  = epoch_iou
            best_model_wts = copy.deepcopy(model.state_dict())
            best_epoch_num = epoch
            
        if phase == "val" and lr_scheduler is not None:
            lr_scheduler.step(epoch_iou)
            
        if phase == "train":
            print(f"{phase} Epoch #{epoch} loss: {train_loss_avg.value} | Best Val IoU: {best_iou}")
        else:
            print(f"{phase} Epoch #{epoch} IoU: {val_iou_avg.value} | Best Val IoU: {best_iou}")
        
        if phase == "val":
            name = str(epoch%2)
            exp_dict = {
                "current_model_wts": copy.deepcopy(model.state_dict()),
                "train_loss_hist": train_loss_hist,
                "val_iou_hist":val_iou_hist,
                "current_optimizer_wts": copy.deepcopy(optimizer.state_dict()), 
                "current_lr_scheduler_wts": copy.deepcopy(lr_scheduler.state_dict()),
                "current_epoch_num": epoch,
                "best_model_wts": best_model_wts,
                "best_epoch_num": best_epoch_num
            }
            torch.save(exp_dict, f"exp_dict_{name}_run1.pth")
            
    time_elapsed = time.time() - since
    print('Epoch complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [ ]:
model.load_state_dict(best_model_wts)

In [ ]:
val_iter  = iter(valid_data_loader)

In [ ]:
images, targets, image_ids = next(val_iter)
images = list(img.to(device) for img in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [ ]:
boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
sample = images[0].permute(1,2,0).cpu().numpy()

In [ ]:
model.eval()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model.eval()
cpu_device = torch.device("cpu")

In [ ]:
sample.shape

In [ ]:
outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

In [ ]:
box = outputs[0]["boxes"][0].detach().numpy().astype(np.int32).tolist()
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

cv2.rectangle(sample,
              (box[0], box[1]),
              (box[2], box[3]),
              (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)